In [5]:
import os
import shutil
import subprocess

def modify_input_file(simulation_number, restart):
    input_content = """\
SIMULATION_TYPE        0
RESTART                {restart}
TEMP                   {temp}
NPART                  108
RHO                    {rho}
R_CUT                  {r_cut}
DELTA                  {delta}
NBLOCKS                {nblocks}
NSTEPS                 {nsteps}

ENDINPUT
"""

    temperature = {1: 1.55, 2: 2, 3: 1}
    rho = {1: 1.1, 2: 0.8, 3: 0.05}
    r_cut = {1: 2.2, 2: 2.5, 3: 5.0}
    delta = {1: 0.001, 2: 0.0005, 3: 0.0005}

    with open("INPUT/input.dat", "w") as f:
        f.write(input_content.format(restart=restart, temp=temperature[simulation_number], rho=rho[simulation_number], r_cut=r_cut[simulation_number], nblocks=100, nsteps=5000, delta=delta[simulation_number]))


def rename_output_files(simulation_number):
    output_folder = "OUTPUT"
    files_to_rename = ["potential_energy.dat", "temperature.dat", "kinetic_energy.dat", "total_energy.dat", "pressure.dat"]
    suffix = ""

    if simulation_number == 1:
        suffix = "_solido"
    elif simulation_number == 2:
        suffix = "_liquido"
    elif simulation_number == 3:
        suffix = "_gassoso"

    for filename in os.listdir(output_folder):
        if filename in files_to_rename:
            old_filepath = os.path.join(output_folder, filename)
            new_filename = f"{filename.split('.')[0]}{suffix}.dat"
            new_filepath = os.path.join(output_folder, new_filename)
            os.rename(old_filepath, new_filepath)


def main():
    # Esegui il codice tramite il makefile
    subprocess.run(["make"], cwd="SOURCE")

    # Simulazioni
    for simulation_number in range(1, 4):
        modify_input_file(simulation_number, restart=0)  # RESTART a 0
        subprocess.run(["./simulator.exe"], cwd="SOURCE")

        # Modifica il file di input per la prossima simulazione
        os.remove("INPUT/CONFIG/config.xyz")
        modify_input_file(simulation_number, restart=1)
        shutil.copy("OUTPUT/CONFIG/config.xyz", "INPUT/CONFIG/config.xyz")
        os.remove("INPUT/CONFIG/velocities.in")
        shutil.move("OUTPUT/CONFIG/velocities.out", "INPUT/CONFIG/velocities.in")

        # Esegui la simulazione
        subprocess.run(["./simulator.exe"], cwd="SOURCE")
        rename_output_files(simulation_number)

        # Rimetti RESTART a 0 e copia config.fcc in config.xyz per la prossima simulazione
        modify_input_file(simulation_number, restart=0)
        os.remove("INPUT/CONFIG/config.xyz")
        shutil.copy("INPUT/CONFIG/config.fcc", "INPUT/CONFIG/config.xyz")


if __name__ == "__main__":
    main()



make: Nothing to be done for 'all'.
